In [23]:
import sys 
sys.path.insert(0,'/hpc/compgen/projects/fragclass/analysis/mvivekanandan/script/madhu_scripts')
import importlib
import random
import numpy as np
import torch
from torch.utils.data import Dataset
import torchvision.transforms.functional as tf
import pandas as pd

import h5py
import config 
import utils

importlib.reload(utils)
importlib.reload(config)

<module 'config' from '/hpc/compgen/projects/fragclass/analysis/mvivekanandan/script/madhu_scripts/config.py'>

In [4]:
#DatasetNames
arguments = {}

arguments["trainingEnformerOutputStoreFile"] = config.filePaths.get("trainingEnformerOutputStoreFile")
arguments["validationEnformerOutputStoreFile"] = config.filePaths.get("validationEnformerOutputStoreFile")

arguments["trainingLabelsDatasetName"] = config.datasetNames.get("trainingLabels")
arguments["validationLabelsDatasetName"] = config.datasetNames.get("validationLabels")
arguments["testLabelsDatasetName"] = config.datasetNames.get("testLabels")
arguments["trainingEnformerOutputDatasetName"] = config.datasetNames.get("trainingEnformerOutput")
arguments["validationEnformerOutputDatasetName"] = config.datasetNames.get("validationEnformerOutput")
arguments["testEnformerOutputDatasetName"] = config.datasetNames.get("testEnformerOutput")

In [26]:
def averageEnformerOutputBins(samples):
    _, numFeatures = samples.shape
    first_bin_end_index = int(numFeatures/2)
    bin_averaged_output = torch.empty((1,first_bin_end_index))
    for sample in samples:
        first_bin_outputs = sample[0:first_bin_end_index].reshape(1, first_bin_end_index)
        second_bin_outputs = sample[first_bin_end_index: numFeatures].reshape(1, first_bin_end_index)
        average = (first_bin_outputs + second_bin_outputs)/2
        bin_averaged_output = torch.cat((bin_averaged_output, average), 0)
        
    return bin_averaged_output[1:, :]

def getMinMeanMax(sampleType):
    with h5py.File(arguments[f"{sampleType}EnformerOutputStoreFile"], 'r') as f:
        enformer_output = f[arguments[f"{sampleType}EnformerOutputDatasetName"]][0:10000]
        encoded_enformer_output = torch.tensor(np.float32(enformer_output))
        averaged = averageEnformerOutputBins(encoded_enformer_output).numpy()
        print(f"Shape of {sampleType} averaged: {averaged.shape}")
        min = np.min(averaged, axis = 0).ravel()
        max = np.max(averaged, axis = 0).ravel()
        mean = np.mean(averaged, axis = 0).ravel()
        print(f"Shapes of {sampleType} min: {min.shape}, max: {max.shape} and mean: {mean.shape}")
        return min, max, mean

# training_min, training_max, training_mean = getMinMeanMax("training")
# validation_min, validation_max, validation_mean = getMinMeanMax("validation")

training_df = pd.DataFrame({'min': training_min, 'max': training_max, "mean": training_mean})
validation_df = pd.DataFrame({"min": validation_min, "max": validation_max, "mean": validation_mean})

training_path = "/hpc/compgen/projects/fragclass/analysis/mvivekanandan/output/EnformerOutputs/training_enformer_track_averages.csv"
validation_path = "/hpc/compgen/projects/fragclass/analysis/mvivekanandan/output/EnformerOutputs/validation_enformer_track_averages.csv"
training_df.to_csv(training_path, sep='\t', index=False)
validation_df.to_csv(validation_path, sep='\t', index=False)